In [ ]:
from ROOT import TFile,TTree,TCanvas,TH1F, gStyle, TLatex, gPad, TLegend, TLorentzVector, TH2F, TLine, TF1, TBox, RDataFrame, TGraphErrors, TFitResultPtr, TF2, TChain
import ROOT
import numpy as np
from math import log10, floor

In [37]:
def round_sig(x, sig=2):
    if(x==0):
        return x
    return round(x, sig-int(floor(log10(abs(x))))-1)

def round_to_reference(x, y):
    if(y==0):
        return x
    return round(x, -int(floor(log10(y))))

def val_and_err(val,err,sig=2):
    err = round_sig(err,sig)
    val = round_to_reference(val,err)
    return val,err

In [1]:
def get_asym_plots(dataframe,cuts,bintype,bins,Mggbins,Mggmin,Mggmax):
    nPlots = len(bins-1)
    hists_total = []
    hists_plus = []
    hists_minus = []
    
    nCuts = len(cuts)
    total_cut = ""
    for i in range(nCuts):
        total_cut += cuts[i]
        total_cut += "&&"
    
    for i in range(nPlots-1):
        final_cut = total_cut + " {} > {:.5f} && {} < {:.5f}".format(bintype,bins[i],bintype,bins[i+1])
        hists_total.append(dataframe.Filter(final_cut).Histo1D(("ht{}".format(i),"",Mggbins,Mggmin,Mggmax),"Mdiphoton"))
        #hists_plus.append(dataframe.Filter(final_cut+"&& helicity == 1").Histo1D(("hp{}".format(i),"",Mggbins,Mggmin,Mggmax),"Mdiphoton"))
        #hists_minus.append(dataframe.Filter(final_cut+"&& helicity == -1").Histo1D(("hm{}".format(i),"",Mggbins,Mggmin,Mggmax),"Mdiphoton"))
    return hists_total#, hists_plus, hists_minus

SyntaxError: invalid syntax (<ipython-input-1-57bd33b43baa>, line 10)

In [1]:
def do_pi0_basic_fit(h,fittype,nfitpars,Mggmin,Mggmax):
    # ========== Find maximum non-zero bin ============
    for i in range(h.GetValue().GetNbinsX()):
        if(h.GetValue().GetBinContent(i)==0):
            Mggmax = h.GetValue().GetBinCenter(i)
            break
    histint=h.Integral()
    h.Scale(1/histint)
    # ========== Create Fit Function ============
    f=TF1("f",fittype,Mggmin,Mggmax,nfitpars)
    f.SetNpx(1000)
    f.SetParameter(0,h.Integral())
    f.SetParameter(1,0.133)
    f.SetParameter(2,0.1)
    for i in range(3,nfitpars):
        f.SetParameter(i,10)
    f.SetParLimits(0,0,1)
    f.SetParLimits(1,0.129,0.135)
    f.SetParLimits(2,0.008,0.1)
    if(fittype=="crystalball(0)+pol2(5)" or fittype=="crystalball(0)+pol4(5)"  ):
        f.SetParLimits(3,-100,100)
        f.SetParLimits(4,-100,100)
    # ========== Perform the fit ============
    h.Fit(f,"Q0 SR")
    h.Fit(f,"Q0 SR")
    # ========== Get residuals plot ============
    bin1 = h.FindBin(Mggmin)
    bin2 = h.FindBin(Mggmax)
    residuals = TGraphErrors(bin2-bin1)
    for i in range(residuals.GetN()):
        BIN = bin1+i
        xval = h.GetBinCenter(BIN)
        yval = h.GetBinContent(BIN)
        yfit = f.Eval(h.GetBinCenter(BIN))
        yerr = h.GetBinError(BIN)
        if(yerr==0):
            continue
        residuals.SetPoint(i,xval,(yfit-yval)/yerr)
        residuals.SetPointError(i,0,0)
    return f,residuals.Clone()

In [1]:
def process_Mgg(hlist,fittype,nfitpars,Mggmin,Mggmax):
    fits = []
    ress = []
    for h in hlist:
        h.GetValue().SetLineColor(1)
        h.GetXaxis().SetNdivisions(508)
        h.GetXaxis().SetRangeUser(Mggmin,Mggmax)
        h.GetXaxis().SetLimits(Mggmin,Mggmax)
        h.GetXaxis().SetTitleSize(0)
        h.GetXaxis().SetLabelSize(0)
        h.SetMarkerStyle(20)
        h.SetTitle(";;Counts")
        f,res = do_pi0_basic_fit(h,fittype,nfitpars,Mggmin,Mggmax)
        fits.append(f)
        res.SetMarkerStyle(20)
        res.SetTitle(";M_{#gamma#gamma} [GeV];Residuals")
        res.GetXaxis().SetTitleSize(0.12)
        res.GetXaxis().SetLabelSize(0.12)
        res.GetYaxis().SetTitleSize(0.12)
        res.GetYaxis().SetLabelSize(0.12)
        res.GetYaxis().SetNdivisions(508)
        res.GetXaxis().SetTitleOffset(1)
        res.GetYaxis().SetTitleOffset(0.3)
        res.GetXaxis().SetLimits(Mggmin,Mggmax)
        ress.append(res)
    return fits, ress

In [1]:
def drawFitLatex(latex,x1,y1,ystep,fit,fittype,u):
    fitpars = []
    if(fittype=="crystalball(0)+pol2(5)"):
        fitpars = ["C","#mu","#sigma","#alpha","N","p0","p1","p2"]
    if(fittype=="crystalball(0)+pol4(5)"):
        fitpars = ["C","#mu","#sigma","#alpha","N","p0","p1","p2","p3","p4"]
    if(fittype=="gaus(0)+pol2(3)"):
        fitpars = ["C","#mu","#sigma","p0","p1","p2"]
    if(fittype=="gaus(0)+pol4(3)"):
        fitpars = ["C","#mu","#sigma","p0","p1","p2","p3","p4"]
        
    for i in range(len(fitpars)):
        val,err = val_and_err(fit.GetParameter(i),fit.GetParError(i),sig=2)
        latex.DrawLatexNDC(x1,y1-i*ystep,"{} = {} #pm {} ".format(fitpars[i],
                                                                  val,
                                                                  err
                                                                 ))
    
    latex.DrawLatexNDC(x1-ystep,y1-ystep*(len(fitpars)),"#chi^{2}/ndf = "+"{:.2f}/{:.0f} = {:.2f}".format(fit.GetChisquare(),
                                                                                               fit.GetNDF(),
                                                                                               fit.GetChisquare()/fit.GetNDF()))
    latex.DrawLatexNDC(x1-ystep,y1-ystep*(len(fitpars)+1),"Purity u = {:.3f}".format(u))
    return

In [12]:
def gethists_A_LU(d,cuts,bintype,bins,phihbins,phiRbins,signal_range,sideband_range):
    nPlots = len(bins-1)
    hists_sig_plus = []
    hists_sig_minus = []
    
    hists_bg_plus = []
    hists_bg_minus = []
    
    nCuts = len(cuts)
    total_cut = ""
    for i in range(nCuts):
        total_cut += cuts[i]
        total_cut += "&&"
    
    
    for i in range(nPlots-1):
        final_cut = total_cut + " {} > {:.5f} && {} < {:.5f}".format(bintype,bins[i],bintype,bins[i+1])
        hists_sig_plus.append(d.Filter(final_cut + " && helicity == 1 && Mdiphoton > {} && Mdiphoton < {}".format(signal_range[0],signal_range[1]))
                      .Histo2D(("h1","h1",phihbins,-3.141592,3.141592,phiRbins,-3.141592,3.141592),"phi_h","phi_R"))

        hists_sig_minus.append(d.Filter(final_cut + " && helicity == -1 && Mdiphoton > {} && Mdiphoton < {}".format(signal_range[0],signal_range[1]))
                      .Histo2D(("h2","h2",phihbins,-3.141592,3.141592,phiRbins,-3.141592,3.141592),"phi_h","phi_R"))

        hists_bg_plus.append(d.Filter(final_cut + " && helicity == 1 && Mdiphoton > {} && Mdiphoton < {}".format(sideband_range[0],sideband_range[1]))
                      .Histo2D(("h3","h3",phihbins,-3.141592,3.141592,phiRbins,-3.141592,3.141592),"phi_h","phi_R"))

        hists_bg_minus.append(d.Filter(final_cut + " && helicity == -1 && Mdiphoton > {} && Mdiphoton < {}".format(sideband_range[0],sideband_range[1]))
                      .Histo2D(("h4","h4",phihbins,-3.141592,3.141592,phiRbins,-3.141592,3.141592),"phi_h","phi_R"))
    
    
    return hists_sig_plus, hists_sig_minus, hists_bg_plus, hists_bg_minus

In [1]:
def get_weighted_polarization(chains,bintype,bins,pol,polerr,cuts,signal_range,sideband_range):
    nPlots = len(bins-1)
    nCuts = len(cuts)
    total_cut = ""
    for i in range(nCuts):
        total_cut += cuts[i]
        total_cut += "&&"

    POL_SIG, POL_BG = [],[]
    POLERR_SIG, POLERR_BG = [],[]
    
    # Our goal is to obtain the weighted polarization for each of the Mh, x, or z binnings
    # This is achieved by first counting the number of events in each RDataFrame file location
    #   ex: Fall2018 inbending, Fall2018 outbending, Spring2019 inbending
    # Then, we weigh the polarization in a traditional fashion, calculating a weighted average w/ error
    # Then, we return the weighted polarization and its error for later usage
    

    for i in range(nPlots-1):
        n_sig, n_bg = np.zeros(len(pol)), np.zeros(len(pol))
        for j in range(len(pol)):
            chain=chains[j]
            final_cut = total_cut + " {} > {:.5f} && {} < {:.5f}".format(bintype,bins[i],bintype,bins[i+1])
            n_sig[j] = chain.GetEntries(final_cut + " && Mdiphoton > {} && Mdiphoton < {}".format(signal_range[0],signal_range[1]))
            n_bg[j]  = chain.GetEntries(final_cut + " && Mdiphoton > {} && Mdiphoton < {}".format(sideband_range[0],sideband_range[1]))
        count = 0
        for n in [n_sig,n_bg]:
            A = np.sum(n*pol)
            B = np.sum(n)
            oA = np.sqrt(np.sum(pol*pol*n+n*n*polerr*polerr))
            oB = np.sqrt(np.sum(n))
            P = A/B # Weighted polarization
            oP = np.sqrt((oA/B)**2+(oB*A/B/B)**2) # Error in weighted polarization
            if(count==0):
                POL_SIG.append(P)
                POLERR_SIG.append(oP)
            else:
                POL_BG.append(P)
                POLERR_BG.append(oP)
            count+=1
    
    print(bintype,"weighted polarization calculation complete")
    return POL_SIG, POLERR_SIG, POL_BG, POLERR_BG 

In [2]:
def process_A_LU(uarray,hists,p):
    
    hists_sig_plus = hists[0]
    hists_sig_minus = hists[1]
    hists_bg_plus = hists[2]
    hists_bg_minus = hists[3]
    
    pol_sig = p[0]
    polerr_sig = p[1]
    pol_bg = p[2]
    polerr_bg = p[3]
    
    f_sigbg=TF2("f_sigbg","[0]*sin(2*x-2*y)+[1]*sin(x-y)+[2]*sin(-x+2*y)+[3]*sin(y)+[4]*sin(x)+[5]*sin(2*x-y)+[6]*sin(3*x-2*y)",-3.141592,3.141592,-3.141592,3.141592)
    f_bg=TF2("f_bg","[0]*sin(2*x-2*y)+[1]*sin(x-y)+[2]*sin(-x+2*y)+[3]*sin(y)+[4]*sin(x)+[5]*sin(2*x-y)+[6]*sin(3*x-2*y)",-3.141592,3.141592,-3.141592,3.141592)
    
    
    fitparams_sigbg = np.zeros((len(uarray),7))
    fiterrors_sigbg = np.zeros((len(uarray),7))
    
    fitparams_sig = np.zeros((len(uarray),7))
    fiterrors_sig = np.zeros((len(uarray),7))
    
    fitparams_bg = np.zeros((len(uarray),7))
    fiterrors_bg = np.zeros((len(uarray),7))
    
    for i in range(len(uarray)):
        hsp1 = hists_sig_plus[i].GetValue().Clone()
        hsp2 = hists_sig_plus[i].GetValue().Clone()
        hsm = hists_sig_minus[i].GetValue().Clone()
        hbp1 = hists_bg_plus[i].GetValue().Clone()
        hbp2 = hists_bg_plus[i].GetValue().Clone()
        hbm = hists_bg_minus[i].GetValue().Clone()
        
        # Set Sumw2 for proper error propagation when dividing
        hsp1.Sumw2()
        hsp2.Sumw2()
        hsm.Sumw2()
        hbp1.Sumw2()
        hbp2.Sumw2()
        hbm.Sumw2()
        
        # For the sig+bg range (N+ - N-)/(N+ + N-)
        hsp1.Add(hsm,-1)
        hsp2.Add(hsm,+1)
        hsp1.Divide(hsp2)
               # Divide by weighted polarization
        h_sig = scale_by_polarization(hsp1,pol_sig[i],polerr_sig[i])
        
        h_sig.Fit(f_sigbg,"SR Q0")
        fitparams_sigbg[i] = np.array([f_sigbg.GetParameter(i) for i in range(7)])
        fiterrors_sigbg[i] = np.array([f_sigbg.GetParError(i) for i in range(7)])
        
        # For the bg range (N+ - N-)/(N+ + N-)
        hbp1.Add(hbm,-1)
        hbp2.Add(hbm,+1)
        hbp1.Divide(hbp2)
            # Divide by weighted polarization
        h_bg = scale_by_polarization(hbp1,pol_bg[i],polerr_bg[i])
        
        h_bg.Fit(f_bg,"SR Q0")
        fitparams_bg[i] = np.array([f_bg.GetParameter(i) for i in range(7)])
        fiterrors_bg[i] = np.array([f_bg.GetParError(i) for i in range(7)])
        
        # From equation 2 on page 10 of the pi+pi0 analysis note
        u=uarray[i]
        fitparams_sig[i] = (1.0/u)*fitparams_sigbg[i] - ((1.0-u)/(u))*fitparams_bg[i]
        fiterrors_sig[i] = np.sqrt( (1.0/u)**2 * fiterrors_sigbg[i]**2 +  ((1.0-u)/(u)) **2 * fiterrors_bg[i]**2   )
    return fitparams_sigbg, fiterrors_sigbg, fitparams_bg, fiterrors_bg, fitparams_sig, fiterrors_sig

In [ ]:
def scale_by_polarization(hist,pol,polerr):
    for i in range(hist.GetNbinsX()):
        for j in range(hist.GetNbinsY()):
            y = hist.GetBinContent(i+1,j+1)
            yerr = hist.GetBinError(i+1,j+1)
            hist.SetBinContent(i+1,j+1,y/pol)
            hist.SetBinError(i+1,j+1,np.sqrt((yerr/pol)**2+(y*polerr/pol/pol)**2))
    return hist

In [4]:
mods = ["sin(2#phi_{h}-2#phi_{R})","sin(#phi_{h}-#phi_{R})", "sin(-#phi_{h}+2#phi_{R})" , "sin(#phi_{R})", "sin(#phi_{h})", "sin(2#phi_{h}-#phi{R})" , "sin(3#phi_{h}-2#phi_{R})"]

In [11]:
def get_u(method,h,fit,fittype,signal_range):
    # method : 0, 1, 2, 3: See Chris' pi0pi+ analysis note
    f_sig = 0
    f_bg = 0
    if(fittype=="crystalball(0)+pol2(5)"):
        f_sig = TF1("f_sig","crystalball",signal_range[0],signal_range[1])
        f_bg = TF1("f_bg","pol2",signal_range[0],signal_range[1])
        [f_sig.SetParameter(i,fit.GetParameter(i)) for i in range(5)]
        [f_bg.SetParameter(i,fit.GetParameter(i+5)) for i in range(3)]
    if(fittype=="crystalball(0)+pol4(5)"):
        f_sig = TF1("f_sig","crystalball",signal_range[0],signal_range[1])
        f_bg = TF1("f_bg","pol4",signal_range[0],signal_range[1])
        [f_sig.SetParameter(i,fit.GetParameter(i)) for i in range(5)]
        [f_bg.SetParameter(i,fit.GetParameter(i+5)) for i in range(5)]
    if(fittype=="gaus(0)+pol2(3)"):
        f_sig = TF1("f_sig","gaus",signal_range[0],signal_range[1])
        f_bg = TF1("f_bg","pol2",signal_range[0],signal_range[1])
        [f_sig.SetParameter(i,fit.GetParameter(i)) for i in range(3)]
        [f_bg.SetParameter(i,fit.GetParameter(i+3)) for i in range(3)]
    if(fittype=="gaus(0)+pol4(3)"):
        f_sig = TF1("f_sig","gaus",signal_range[0],signal_range[1])
        f_bg = TF1("f_bg","pol4",signal_range[0],signal_range[1])
        [f_sig.SetParameter(i,fit.GetParameter(i)) for i in range(3)]
        [f_bg.SetParameter(i,fit.GetParameter(i+3)) for i in range(5)]
    

    total_hist_sum = h.Integral(h.FindBin(signal_range[0]), h.FindBin(signal_range[1]))
    
    sig_fit_integral = f_sig.Integral(signal_range[0],signal_range[1])/h.GetBinWidth(1)
    bg_fit_integral = f_bg.Integral(signal_range[0],signal_range[1])/h.GetBinWidth(1)
    total_fit_integral = sig_fit_integral + bg_fit_integral
    
    u = 0
    if(method==0):
        u = sig_fit_integral/total_hist_sum
    elif(method==1):
        u = sig_fit_integral/total_fit_integral
    elif(method==2):
        u = 1 - bg_fit_integral/total_hist_sum
    elif(method==3):
        u = 1 - bg_fit_integral/total_fit_integral
    
    return u

In [ ]:
def get_asym_graph(bins,fitpars,errors,color,marker,plotyranges,idx_mods,plotstr):
    tges=[]

    for i in range(len(idx_mods)):

        idx = idx_mods[i]
        modulation_name = mods[idx]

        # Create TGraphErrors for plotting
        tges.append(TGraphErrors(len(bins)-1))
        tges[i].SetMarkerStyle(marker)
        tges[i].SetMarkerColor(color)
        tges[i].SetLineColor(color)
        tges[i].SetTitle("{}".format(modulation_name)+";{};".format(plotstr))

        for j in range(tges[i].GetN()):
            x = np.mean(bins[j:j+2])
            y = fitpars[j,idx]
            yerr = errors[j,idx]
            tges[i].SetPoint(j,x,y)
            tges[i].SetPointError(j,0,yerr)
            tges[i].GetYaxis().SetRangeUser(plotyranges[i][0],plotyranges[i][1])
    return tges